# CNN on Road Sign Classification

This notebook contains a CNN which classifies 4 types of road signs.

In [ ]:
# import libraries
import numpy as np
import random
import cv2
import os
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.optimizers import SGD, Adam

### Loading data

In [ ]:
# this function loads the images and create training data
labels=['1','2','3','4']
DATADIR = './RoadSign'
def create_training_data(training_set, img_height, img_width):
    for label in labels:
        path=os.path.join(DATADIR, label) # get dir of the folder for the current class
        category = int(label)-1 # index starts from 0
        for img in os.listdir(path): # for each image
            img_array = cv2.imread(os.path.join(path, img)) # read the image as 3-d array
            img_array = cv2.resize(img_array, (img_height, img_width)) # resize the image to the min size
            training_set.append([img_array, category]) # add this image to the training set

In [ ]:
# load the sizes of all the images to compute the minimum height and width
import PIL
width_list = []
height_list = []
os.chdir('./data')
for i in range(1,5):
    os.chdir("./{num}".format(num=i))
    for file in os.listdir():
        image = PIL.Image.open(file)
        width, height = image.size
        width_list.append(width)
        height_list.append(height)
    os.chdir('..')
os.chdir('..')
# compute the smallest height and width
width = np.min(width_list)
height = np.min(height_list)
IMG_SIZE = np.min([width, height])
print('IMG_SIZE=', IMG_SIZE)

In [ ]:
# create the training set
training_data = []            
create_training_data(training_data, IMG_SIZE, IMG_SIZE)
random.shuffle(training_data) # randomly shuffle the training pais
X = [] # features
y = [] # labels
for img, label in training_data:
    X.append(img)
    y.append(label)
X = np.array(X) # feature array
y = np.array(y).reshape(-1,1) # label array
y = keras.utils.to_categorical(y) # convert class number to one-hot encoding
print(np.shape(X), np.shape(y)) # check if their shapes are correct

In [ ]:
# print the images and labels to check if they are loaded correctly
fig, ax = plt.subplots(1,10)
for i in range(10):
    ax[i].imshow(X[i])    
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    print(y[i])

### Defining CNN model

In [ ]:
# Now create the model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.summary()

In [ ]:
# compile the model
model.compile(optimizer=SGD(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 30

# now that we have X and y, we fit the model
model.fit( X, y, batch_size= batch_size, epochs=epochs, validation_split=0.2)